Se realiza el cargue de las librerias que se van a utilizar


In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from urllib.parse import urljoin
import os    
import re
from tqdm import tqdm
import random
from time import sleep
import tkinter as tk
from tkinter import filedialog
from pandas import DataFrame

Se define la función para extraer robots.txt


In [2]:
def obt_robot(siteurl):
    if siteurl.endswith('/'):
        path=siteurl
    else:
        path=siteurl+'/'
    req=requests.get(path+'robots.txt',data=None)
    return req.text

In [3]:
text = obt_robot('https://www.soriana.com/')

In [4]:
data = re.sub( r'<[^>]*>', ' ', text).strip()

print(data)

User-agent: * 
Allow: /&nbsp; 

 Disallow: /soriana/es/cart 
Disallow: /soriana/es/checkout 
Disallow: /soriana/es/my-account 
Disallow: /combined.css* 
Disallow: /combined.js* 
Disallow: /*/quickView 
Disallow: /*/zoomImages 

 Sitemap: https://www.soriana.com/sitemap.xml 

 User-agent: CazoodleBot 
Disallow: / 

 User-agent: MJ12bot 
Disallow: / 

 # Block dotbot as it cannot parse base urls properly 
User-agent: dotbot/1.0 
Disallow: / 

 # Block Gigabot 
User-agent: Gigabot 
Disallow: / 

 # Block&nbsp; 
Disallow: /*/TarjetaVisaLink 
Disallow: /*/TarjetaMastercardLink 
Disallow: /*/TarjetaAprecioLink 
Disallow: /*/TarjetaPaypalLink 
Disallow: /*/TarjetaBBVaLink 
Disallow: /*/TarjetaVisaCheckoutLink


Como se aprecia el fichero robots.txt nos muestra los sitios que no deberíamos tomar en cuenta para nuestra extracción y que se respetarán durante la ejecución de este proyecto.

In [5]:
#pip install builtwith

In [6]:
#Identificamos las tecnologías presentes

import builtwith
#tech_used = builtwith.parse('https://www.soriana.com/')
#print(tech_used)

In [7]:
#pip3 install python-whois

In [8]:
#Buscando datos sobre el dominio
import whois
whois.whois('soriana.com/')

{'domain_name': 'SORIANA.COM',
 'registrar': 'Domain.com, LLC',
 'whois_server': 'whois.domain.com',
 'referral_url': None,
 'updated_date': datetime.datetime(2020, 10, 6, 20, 29, 5),
 'creation_date': datetime.datetime(2000, 11, 1, 16, 36, 39),
 'expiration_date': datetime.datetime(2025, 11, 1, 16, 36, 39),
 'name_servers': ['NS1.KIONETWORKS.NET',
  'NS2.KIONETWORKS.NET',
  'NS3.KIONETWORKS.NET',
  'ns1.kionetworks.net',
  'ns2.kionetworks.net',
  'ns3.kionetworks.net'],
 'status': 'clientDeleteProhibited https://icann.org/epp#clientDeleteProhibited',
 'emails': ['compliance@domain-inc.net', 'support@dotster-inc.com'],
 'dnssec': 'unsigned',
 'name': 'REDACTED FOR PRIVACY',
 'org': 'REDACTED FOR PRIVACY',
 'address': 'REDACTED FOR PRIVACY',
 'city': 'REDACTED FOR PRIVACY',
 'state': 'Nuevo Leon',
 'zipcode': 'REDACTED FOR PRIVACY',
 'country': 'MX'}

In [9]:
#Se generan una serie de listas para guardar información correspondiente.
#Nombre de los productos
Nprod=[]
#Precios Original de los productos
POriginal=[]
#Precios en rebaja de los productos
PRebaja=[]
#Guarda las urls de las imagenes.
imgurls=[]
#Las siguientes variables se pueden utilizar cuando se hace clic en el producto
#Puntuaciones de los productos
Puntuaciones=[]
#Reseñas de los productos
Resena=[]
#Detalles generales
Detalles=[]


Se define la URl del sitio escogido, la cadena de supermercados Soriana.
También se genera el user agent en el cual puede contener detalles como el tipo de aplicación, sistema operativo, proveedor de software o la versión de software.
El cual se utiliza para realizar la solicitud al sitio web.
Y poder extraer la información de la págna teniendo en cuenta el lenguaje html.


Nota: Si el siguiente bloque de código parece no ejecutarse se recomienda cambiar/alternar el User-Agent.


In [10]:
#agent = {"User-Agent": 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36'}


In [11]:
agent= {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36','Accept-Encoding': 'gzip, deflate, br','Accept-Language': 'en-US,en;q=0.9,hi;q=0.8'}

In [15]:
agent2 = {
"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,\*/*;q=0.8",
"Accept-Encoding": "gzip, deflate, sdch, br",
"Accept-Language": "en-US,en;q=0.8",
"Cache-Control": "no-cache",
"dnt": "1",
"Pragma": "no-cache",
"Upgrade-Insecure-Requests": "1",
"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) AppleWebKit/5\
37.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36"
}

In [16]:
URL='https://www.soriana.com/'
#agent = {"User-Agent":'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'}
page = requests.get(URL, headers=agent)
c=page.content
soup=BeautifulSoup(c,'html')


Se realiza una busqueda de componentes con la etiqueta li, donde se realiza un filtro para aquellos que no contienen información
Se generan dos listas para guardar información de las subcategorias y los links de la página web, mediante la etiqueta title 
y href la cual se llena mediante el ciclo for.


In [17]:
summary2=soup.findAll('li',{'class':'yCmsComponent Lc right_col'})
a=summary2[40]
summary2=[x for x in summary2 if x!=a]
subtitulos=[]
sublinks=[]

for j in summary2:
    subtitulos.append(j.find('a').get('title'))
    sublinks.append(j.find('a').get('href'))

Algunas de las urls que se han extraído un guardado en sublinks, no se
encuentran completas, es decir hace falta la parte principal "https://www.soriana.com/"
por lo cual se realiza el ajuste correspondiente con un ciclo for.



In [18]:
for i in range(0,len(sublinks)):
    if 'https' not in sublinks[i]:
        sublinks[i]=urljoin(URL, sublinks[i])


Se genera un dataframe para guardar la información del nombre de las subcategorias
y de los links correspondientes.


In [19]:
df=pd.DataFrame({'Subcategorias':subtitulos,'Sublinks':sublinks},
                columns=['Subcategorias','Sublinks'])
df=df[29:]

In [20]:
#df.to_csv('C:/David/UOC/Sem 2/Tipo/PRA1/notebooks/sublinks.csv', index = False, header=True)

In [21]:
print(df)

                Subcategorias  \
29                  Pantallas   
30     Soportes para pantalla   
31   Accesorios de tv y video   
32                  Streaming   
33             Teatro en casa   
..                        ...   
488                  Plafones   
489        Lámparas colgantes   
490                 Linternas   
491        Bohemia Cristalina   
492                   Carlyle   

                                              Sublinks  
29   https://www.soriana.com/soriana/es/c/Electr%C3...  
30   https://www.soriana.com/soriana/es/c/Electr%C3...  
31   https://www.soriana.com/soriana/es/c/Electr%C3...  
32   https://www.soriana.com/soriana/es/c/Electr%C3...  
33   https://www.soriana.com/soriana/es/c/Electr%C3...  
..                                                 ...  
488  https://www.soriana.com/soriana/es/c/Ferreter%...  
489  https://www.soriana.com/soriana/es/c/Ferreter%...  
490  https://www.soriana.com/soriana/es/c/Ferreter%...  
491  https://www.soriana.com/sori


Se realiza el mismo paso anterior: donde se extrae la información de 
cada una de las URLs que se han guardado en el dataframe. En este caso, 
nos interesa el extraer la información correspondientes a los items que
se encuentran dentro de cada una de las subcategorias.


In [22]:
newURL=df.iat[0,1]
page = requests.get(newURL, headers=agent)
c=page.content
soup=BeautifulSoup(c,'html')
summary=soup.find('div',{'class':"product__listing product__grid"})
land_list=summary.findAll('div',{"class": "product-item"})

En este caso se tiene una variación en las etiquetas para encontrar el
precio como el nombre del producto, por lo cual se realiza el manejo de
excepciones correspondiente a cada cambio de etiqueta entre el precio y 
el nombre del producto. Igualmente se realiza la extracción de las URLs
del material audiovisual.

In [23]:
#Imprimo el tiempo de inicio de la extracción
import time
t = time.localtime()
current_time = time.strftime("%H:%M:%S", t)
print(current_time)

21:15:46


En caso de que el servidor no responda se debe usar la función 'sleep' que se encuentra comentada 

In [24]:
for k in range(0,464):
    newURL=df.iat[k,1]
    page = requests.get(newURL, headers=agent)
    c=page.content
    soup=BeautifulSoup(c,'html')
    summary=soup.find('div',{'class':"product__listing product__grid"})
    try:
        land_list=summary.findAll('div',{"class": "product-item"})
    except AttributeError:
        try:
            summary=soup.find('div',{'class':"carousel__component--carousel js-owl-carousel js-owl-default"})
            land_list=summary.findAll('div',{"class": "carousel__item"})       
        except AttributeError:
            pass
    for land in land_list:
        try:
            # espero que cargue y se almacene la información
            #sleep(random.uniform(1.0, 5.0))
            Nprod.append(land.find('a',{'class':'thumb'}).get('title'))
        except AttributeError:
            Nprod.append(land.find('div',{'class':'carousel__item--name'}).text)
        a=land.find('div',{'class':'priceContainer'})
        try:
            #sleep(random.uniform(1.0, 5.0))
            POriginal.append(a.find('span',{'class':'sale-original-price'}).text)
        except AttributeError:
            try:
             #   sleep(random.uniform(1.0, 5.0))
                POriginal.append(a.find('span',{'class':'price'}).text)
            except AttributeError:
                POriginal.append(a.find('p',{'class':'price'}).text)
        try:
            #sleep(random.uniform(1.0, 5.0))
            PRebaja.append(a.find('span',{'class':'price sale-price'}).text)
        except AttributeError:
            PRebaja.append('NA')
        try:
            #sleep(random.uniform(1.0, 5.0))
            img_url=land_list[0].find('img',{'id':'prodDetailsImgZoom'}).get('src')
        except AttributeError:
            img_url=land_list[0].find('img',{'class':'item lazyOwl'}).get('src')
        if not img_url:
            continue
        img_url = urljoin(newURL, img_url)
        try:
            sleep(random.uniform(4.0, 6.0))
            pos = img_url.index("?")
            img_url = img_url[:pos]
        except ValueError:
            pass
        imgurls.append(img_url)

ConnectionError: HTTPSConnectionPool(host='www.soriana.com', port=443): Max retries exceeded with url: /soriana/es/c/Videojuegos/XBOX/Accesorios-XBOX/c/E13 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000005B31307A08>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [25]:
#Compruebo el tiempo en que finalizó la extracción 
t = time.localtime()
current_time = time.strftime("%H:%M:%S", t)
print(current_time)

21:34:19


Se guarda toda la información extraída (Nombre del producto, precio normal y precio de rebaja en caso de encontrarse registrado) en un nuevo dataframe.

In [26]:
dfNew=pd.DataFrame({'Nombre_Producto':Nprod,'Precio_Original':POriginal,'Precio_Rebaja':PRebaja},
                columns=['Nombre_Producto','Precio_Original','Precio_Rebaja'])

In [27]:
dfNew.head()

,Nombre_Producto,Precio_Original,Precio_Rebaja
0,Pantalla Samsung 58 Pulg 4K UHD LED Smart TV,"$18,290.00","$12,490.00"
1,Pantalla Samsung 50 Pulg 4K UHD LED Smart TV,"$14,999.00","$10,690.00"
2,Pantalla Samsung 55 Pulg 4K UHD LED Smart TV,"$17,790.00","$11,790.00"
3,Pantalla Vios 50 plg 4K UHD LED Smart TV,"$10,299.00","$6,490.00"
4,Pantalla Vios 65 plg UHD 4K LED Smart TV,"$16,999.00","$11,490.00"


In [28]:
#Comprobamos cuántas filas se puedieron extraer con éxito
len(dfNew)

203

Se observa que dependiendo del useragent o las veces que se corra el script con/sin delays, los valores que se extraen van a difierir si el servidor bloquea al crawler. 

In [29]:
#GUI para exportar el dataframe a fichero CSV
root= tk.Tk()

canvas1 = tk.Canvas(root, width = 300, height = 300, bg = 'lightsteelblue2', relief = 'raised')
canvas1.pack()

def exportCSV ():
    global df
    
    export_file_path = filedialog.asksaveasfilename(defaultextension='.csv')
    dfNew.to_csv (export_file_path, index = False, header=True)

saveAsButton_CSV = tk.Button(text='Exportar CSV', command=exportCSV, bg='green', fg='white', font=('helvetica', 12, 'bold'))
canvas1.create_window(150, 150, window=saveAsButton_CSV)
root.mainloop()


Se define una función para realizar la correspondiente descarga
del material audiovisual


In [30]:
def download(url, pathname):
    if not os.path.isdir(pathname):
        os.makedirs(pathname)
    response = requests.get(url, stream=True)
    file_size = int(response.headers.get("Content-Length", 0))
    filename = os.path.join(pathname, url.split("/")[-1])
    progress = tqdm(response.iter_content(1024), f"Downloading {filename}", total=file_size, unit="B", unit_scale=True, unit_divisor=1024)
    with open(filename, "wb") as f:
        for data in progress:
            f.write(data)
            progress.update(len(data))

In [31]:
#Para descargar el contenido audiovisual se debe ingresar una ruta de destino local

path = 'C:/Users/David/Pictures/test'

for img in imgurls:
    try:
        download(img, path)
        # C:\Alexandra Gomez\Personal\Máster\Tipología y ciclo de vidas de los datos\Práctica Web Scraping\Imagenes')
    except:
        pass

In [32]:
imgurls

['https://www.soriana.com/medias/sys_master/images/images/he1/h31/9085902258206/8806090312892-A-300x300.jpg',
 'https://www.soriana.com/medias/sys_master/images/images/he1/h31/9085902258206/8806090312892-A-300x300.jpg',
 'https://www.soriana.com/medias/sys_master/images/images/he1/h31/9085902258206/8806090312892-A-300x300.jpg',
 'https://www.soriana.com/medias/sys_master/images/images/he1/h31/9085902258206/8806090312892-A-300x300.jpg',
 'https://www.soriana.com/medias/sys_master/images/images/he1/h31/9085902258206/8806090312892-A-300x300.jpg',
 'https://www.soriana.com/medias/sys_master/images/images/he1/h31/9085902258206/8806090312892-A-300x300.jpg',
 'https://www.soriana.com/medias/sys_master/images/images/he1/h31/9085902258206/8806090312892-A-300x300.jpg',
 'https://www.soriana.com/medias/sys_master/images/images/he1/h31/9085902258206/8806090312892-A-300x300.jpg',
 'https://www.soriana.com/medias/sys_master/images/images/he1/h31/9085902258206/8806090312892-A-300x300.jpg',
 'https://